In [ ]:
!nvidia-smi


Sun Feb  8 10:07:09 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -q \
  torch \
  transformers \
  peft \
  trl \
  accelerate \
  datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 14.9 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

MODEL_NAME = "microsoft/phi-3-mini-4k-instruct"

# Load dataset
dataset = load_dataset("json", data_files="medical_inst.jsonl", split="train")

def format_prompt(example):
    return {
        "text": f"""### Instruction:
{example['instruction']}

### Response:
{example['output']}"""
    }

dataset = dataset.map(format_prompt)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Apply LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Training config
training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    max_steps=250,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
)

trainer.train()


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Adding EOS to train dataset:   0%|          | 0/53 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/53 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/53 [00:00<?, ? examples/s]

Step,Training Loss
10,1.278267
20,0.813248
30,0.546291
40,0.377338
50,0.248632
60,0.156628
70,0.130155
80,0.117257
90,0.110998
100,0.107410


TrainOutput(global_step=250, training_loss=0.21390988683700563, metrics={'train_runtime': 294.4842, 'train_samples_per_second': 6.792, 'train_steps_per_second': 0.849, 'total_flos': 1844065952409600.0, 'train_loss': 0.21390988683700563})

In [ ]:
model.save_pretrained(
    "phi3_medical_lora_bin",
    safe_serialization=False
)

print("✅ LoRA adapter saved as .bin")


✅ LoRA adapter saved as .bin


In [ ]:
!ls -lh phi3_medical_lora_bin


total 35M
-rw-r--r-- 1 root root 1.1K Feb  8 10:17 adapter_config.json
-rw-r--r-- 1 root root  35M Feb  8 10:17 adapter_model.bin
-rw-r--r-- 1 root root 5.2K Feb  8 10:17 README.md


In [ ]:
!zip -r phi3_medical_lora_bin.zip phi3_medical_lora_bin


  adding: phi3_medical_lora_bin/ (stored 0%)
  adding: phi3_medical_lora_bin/adapter_config.json (deflated 58%)
  adding: phi3_medical_lora_bin/README.md (deflated 65%)
  adding: phi3_medical_lora_bin/adapter_model.bin (deflated 7%)


In [ ]:
!ls -lh phi3_medical_lora_bin.zip


-rw-r--r-- 1 root root 32M Feb  8 10:21 phi3_medical_lora_bin.zip
